In [13]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
   text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [14]:
# tokenize the chars
string_to_int = { ch: i for i, ch in enumerate(chars)} # create a dictionary of the characters
int_to_string = {i:ch for i, ch in enumerate(chars)} # create a reverse dictionary of the characters
encode = lambda words: [string_to_int[char] for char in words]
decode = lambda chars: ''.join([int_to_string[char] for char in chars])

encode_hello = encode('hello')
print(encode_hello)
decoded_hello = decode(encode_hello)
print(decoded_hello)


[61, 58, 65, 65, 68]
hello


In [15]:
import torch

### Using batches to accelerate training process


In [16]:
# check if cuda is available in our hardware
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8 # length of the sequence
batch_size = 4 # number of blocks we are processing in parallel


cpu


In [17]:
data = torch.tensor(encode(text), dtype=torch.long) # convert the text to a tensor
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


## Validation and Training Split


In [18]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

In [19]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
   context = x[:t+1]
   target = y[t]
   print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
